In [29]:
import sys

sys.path.append('../')
from auton_survival import datasets
outcomes, features = datasets.load_support()

In [30]:
outcomes

,event,time
0,0,2029
1,1,4
2,1,47
3,1,133
4,0,2029
...,...,...
9100,0,350
9101,0,347
9102,0,346
9103,1,7


In [31]:
features

,sex,dzgroup,dzclass,income,race,ca,age,num.co,meanbp,wblc,...,alb,bili,crea,sod,ph,glucose,bun,urine,adlp,adls
0,male,Lung Cancer,Cancer,$11-$25k,other,metastatic,62.84998,0,97.0,6.000000,...,1.799805,0.199982,1.199951,141.0,7.459961,NaN,NaN,NaN,7.0,7.0
1,female,Cirrhosis,COPD/CHF/Cirrhosis,$11-$25k,white,no,60.33899,2,43.0,17.097656,...,NaN,NaN,5.500000,132.0,7.250000,NaN,NaN,NaN,NaN,1.0
2,female,Cirrhosis,COPD/CHF/Cirrhosis,under $11k,white,no,52.74698,2,70.0,8.500000,...,NaN,2.199707,2.000000,134.0,7.459961,NaN,NaN,NaN,1.0,0.0
3,female,Lung Cancer,Cancer,under $11k,white,metastatic,42.38498,2,75.0,9.099609,...,NaN,NaN,0.799927,139.0,NaN,NaN,NaN,NaN,0.0,0.0
4,female,ARF/MOSF w/Sepsis,ARF/MOSF,NaN,white,no,79.88495,1,59.0,13.500000,...,NaN,NaN,0.799927,143.0,7.509766,NaN,NaN,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9100,male,ARF/MOSF w/Sepsis,ARF/MOSF,NaN,white,no,66.07300,1,109.0,7.399414,...,3.699707,0.399963,1.099854,131.0,7.459961,188.0,21.0,NaN,NaN,0.0
9101,female,Coma,Coma,NaN,white,no,55.15399,1,43.0,NaN,...,NaN,NaN,5.899414,135.0,7.289062,190.0,49.0,0.0,NaN,0.0
9102,male,ARF/MOSF w/Sepsis,ARF/MOSF,NaN,white,no,70.38196,1,111.0,8.398438,...,NaN,0.399963,2.699707,139.0,7.379883,189.0,60.0,3900.0,NaN,NaN
9103,male,MOSF w/Malig,ARF/MOSF,NaN,white,yes,47.01999,1,99.0,7.599609,...,1.199951,0.399963,3.500000,135.0,7.469727,246.0,55.0,NaN,NaN,0.0


In [38]:
from auton_survival import datasets, preprocessing, models

cat_feats = ['sex', 'dzgroup', 'dzclass', 'income', 'race', 'ca']
num_feats = ['age', 'num.co', 'meanbp', 'wblc', 'hrt', 'resp',
             'temp', 'pafi', 'alb', 'bili', 'crea', 'sod', 'ph',
             'glucose', 'bun', 'urine', 'adlp', 'adls']
# Preprocess (Impute and Scale) the features
features = preprocessing.Preprocessor().fit_transform(features, cat_feats, num_feats)

# Train a Deep Cox Proportional Hazards (DCPH) model
model = models.cph.DeepCoxPH(layers=[100])
model.fit(features, outcomes.time, outcomes.event)



100%|██████████| 1/1 [00:00<00:00, 12.27it/s]


In [41]:
# Predict risk at specific time horizons.
predictions = model.predict_risk(features.iloc[0], t=[8, 12, 16])

In [42]:
predictions

array([[0.20516453, 0.28220015, 0.33294449]])

In [3]:
from auton_survival.preprocessing import Preprocessor

cat_feats = ['sex', 'dzgroup', 'dzclass', 'income', 'race', 'ca']
num_feats = ['age', 'num.co', 'meanbp', 'wblc', 'hrt', 'resp',
             'temp', 'pafi', 'alb', 'bili', 'crea', 'sod', 'ph',
             'glucose', 'bun', 'urine', 'adlp', 'adls']

# Data should be processed in a fold-independent manner when performing cross-validation.
# For simplicity in this demo, we process the dataset in a non-independent manner.
preprocessor = Preprocessor(cat_feat_strat='ignore', num_feat_strat= 'mean')
x = preprocessor.fit_transform(features, cat_feats=cat_feats, num_feats=num_feats,
                               one_hot=True, fill_value=-1)

In [4]:
import numpy as np
times = np.quantile(outcomes.time[outcomes.event==1], [0.25, 0.5, 0.75]).tolist()

In [5]:
from auton_survival.experiments import SurvivalRegressionCV

param_grid = {'k' : [3],
              'distribution' : ['Weibull'],
              'learning_rate' : [1e-4, 1e-3],
              'layers' : [[100]]}

experiment = SurvivalRegressionCV(model='dsm', num_folds=3, hyperparam_grid=param_grid, random_seed=0)
model = experiment.fit(x, outcomes, times, metric='brs')

At hyper-param {'distribution': 'Weibull', 'k': 3, 'layers': [100], 'learning_rate': 0.0001}
At fold: 0


100%|██████████| 50/50 [00:04<00:00, 11.33it/s]


At fold: 1


100%|██████████| 50/50 [00:04<00:00, 11.18it/s]


At fold: 2


...

100%|██████████| 50/50 [00:04<00:00, 11.44it/s]


At fold: 1


100%|██████████| 50/50 [00:04<00:00, 11.40it/s]


At fold: 2


100%|██████████| 50/50 [00:06<00:00,  7.48it/s]


In [6]:
print(experiment.folds)
model

[2 2 0 ... 0 0 0]


In [7]:
out_risk = model.predict_risk(x, times)
out_survival = model.predict_survival(x, times)

In [8]:
from auton_survival.metrics import survival_regression_metric

for fold in set(experiment.folds):
    print(survival_regression_metric('brs', outcomes[experiment.folds==fold],
                                     out_survival[experiment.folds==fold],
                                     times=times))

[0.12917649 0.19436242 0.20631976]
[0.12691664 0.19280962 0.20635106]
[0.12159038 0.19158577 0.20871653]


/Users/davidlee/Documents/GitHub/DeepSurvivalMachines/auton_survival/metrics.py:212: UserWarning: You are are evaluating model performance on the same data used to estimate the censoring distribution.
  warnings.warn("You are are evaluating model performance on the \
/Users/davidlee/Documents/GitHub/DeepSurvivalMachines/auton_survival/metrics.py:212: UserWarning: You are are evaluating model performance on the same data used to estimate the censoring distribution.
  warnings.warn("You are are evaluating model performance on the \
/Users/davidlee/Documents/GitHub/DeepSurvivalMachines/auton_survival/metrics.py:212: UserWarning: You are are evaluating model performance on the same data used to estimate the censoring distribution.
  warnings.warn("You are are evaluating model performance on the \


In [9]:
from auton_survival.metrics import survival_regression_metric

for fold in set(experiment.folds):
    print(survival_regression_metric('ctd', outcomes[experiment.folds==fold],
                                     out_survival[experiment.folds==fold],
                                     times=times))

/Users/davidlee/Documents/GitHub/DeepSurvivalMachines/auton_survival/metrics.py:212: UserWarning: You are are evaluating model performance on the same data used to estimate the censoring distribution.
  warnings.warn("You are are evaluating model performance on the \


[0.765472457063747, 0.7223859858263633, 0.688765174086125]


/Users/davidlee/Documents/GitHub/DeepSurvivalMachines/auton_survival/metrics.py:212: UserWarning: You are are evaluating model performance on the same data used to estimate the censoring distribution.
  warnings.warn("You are are evaluating model performance on the \


[0.781271531344058, 0.7275326444323827, 0.688923316104416]


/Users/davidlee/Documents/GitHub/DeepSurvivalMachines/auton_survival/metrics.py:212: UserWarning: You are are evaluating model performance on the same data used to estimate the censoring distribution.
  warnings.warn("You are are evaluating model performance on the \


[0.7612520620726958, 0.7181912109861889, 0.6816684343816966]


In [10]:
for fold in set(experiment.folds):
    for time in times:
        print(time)

14.0
58.0
252.0
14.0
58.0
252.0
14.0
58.0
252.0
